# Imports

In [1]:
import sqlite3
from prettytable import from_db_cursor
from project_paths import EMPLOYEES_DB

# Connect to database

In [2]:
con = sqlite3.connect(EMPLOYEES_DB)
cur = con.cursor()

# Source table

In [3]:
cur.execute("""
    SELECT      *
    FROM        tEmployeeList
""")

In [4]:
res = from_db_cursor(cur)
res.align = "l"
print(res)

+------------+--------------+-----------------------------+-----------+
| EmployeeId | EmployeeName | EmployeeTitle               | ManagerId |
+------------+--------------+-----------------------------+-----------+
| 1          | John T.      | Director of American Office | None      |
| 2          | George F.    | Manager                     | 1         |
| 3          | Elliot S.    | Driver                      | 2         |
| 11         | Anna K.      | Director of European Office | None      |
| 12         | Elsa F.      | Senior Manager              | 11        |
| 13         | Olaf S.      | Snowma                      | 12        |
| 21         | Tom K.       | Cat in European Office      | 11        |
| 22         | Kristoff I.  | Ice Manager                 | 21        |
| 23         | Troll K.     | Stone Manager               | 22        |
+------------+--------------+-----------------------------+-----------+


# Solution

In [5]:
cur.execute("""
WITH t1 AS (
    SELECT  EmployeeId,
            EmployeeName,
            EmployeeTitle,
            IFNULL(ManagerId, EmployeeId) as ManagerId
    FROM    tEmployeeList
),

t2 AS (
        SELECT          t1.EmployeeId, t1.EmployeeName, t1.EmployeeTitle, t1.ManagerId,
                        b.EmployeeName as ManagerName,
                        c.EmployeeId as ManagersManagerId,
                        c.EmployeeName as ManagersManagerName,
                        d.EmployeeId as DirectorId,
                        d.EmployeeName as DirectorName,
                        CASE
                                WHEN t1.ManagerId = t1.EmployeeId THEN "l1"
                                WHEN t1.ManagerId = d.EmployeeId THEN "l2"
                                WHEN c.EmployeeId = d.EmployeeId THEN "l3"
                                ELSE "l4"
                                END as level
        FROM            t1
        LEFT JOIN       t1 as b
                ON      t1.ManagerId = b.EmployeeId
        LEFT JOIN       t1 as c
                ON      b.ManagerId = c.EmployeeId
        LEFT JOIN       t1 as d
                ON      c.ManagerId = d.EmployeeId
)

SELECT      EmployeeId,
            EmployeeName,
            EmployeeTitle,
            ManagerId,
            ManagerName,
            DirectorName,
            CASE
                WHEN level  = "l1" THEN EmployeeName
                WHEN level  = "l2" THEN ManagerName || " | " || EmployeeName
                WHEN level  = "l3" THEN DirectorName || " | " || ManagerName || " | " || EmployeeName
                ELSE DirectorName || " | " || ManagerName || " | " || ManagersManagerName || " | " || EmployeeName
                END as PositionBreadcrumbs
FROM        t2

""")

In [6]:
res = from_db_cursor(cur)
res.align = "l"
print(res)

+------------+--------------+-----------------------------+-----------+-------------+--------------+-------------------------------------------+
| EmployeeId | EmployeeName | EmployeeTitle               | ManagerId | ManagerName | DirectorName | PositionBreadcrumbs                       |
+------------+--------------+-----------------------------+-----------+-------------+--------------+-------------------------------------------+
| 1          | John T.      | Director of American Office | 1         | John T.     | John T.      | John T.                                   |
| 2          | George F.    | Manager                     | 1         | John T.     | John T.      | John T. | George F.                       |
| 3          | Elliot S.    | Driver                      | 2         | George F.   | John T.      | John T. | George F. | Elliot S.           |
| 11         | Anna K.      | Director of European Office | 11        | Anna K.     | Anna K.      | Anna K.                      

# Close connection to database

In [7]:
cur.close()